<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [7]:
import numpy as np
import string

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
corpus = corpus.reshape(3,1)
docu =[]

for doc in corpus:
    for i in doc:
        term = str(i)
        term = term.split(' ')
        docu.append(term)

docu = np.array((docu), dtype = object)
print(f'La lista de documentos es: {docu}')
print(100* '*')

flatten_terms = [element for sublist in docu for element in sublist]
terms = set(flatten_terms)

unique_terms = []

for i in terms:
    unique_terms.append(i)

vocab = np.array(unique_terms)
print(f'el vector de términos únicos es: {vocab}')



La lista de documentos es: [list(['que', 'dia', 'es', 'hoy'])
 list(['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'])
 list(['martes', 'muchas', 'gracias'])]
****************************************************************************************************
el vector de términos únicos es: ['que' 'de' 'es' 'gracias' 'hoy' 'el' 'dia' 'muchas' 'martes']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [197]:
import string

def ohe(doc, vocab):
    
    words_index={}
    
    for word in vocab:
        word_no_punctuation = word.strip(string.punctuation)
        words_index[word_no_punctuation] = len(words_index)
    print(f'El index para cada término del vocabulario es: {words_index}')
    
    integer_encoded =[words_index[words] for words in doc]
    print(f'los términos del documento son: {doc}')
    print(f'Los indexes de los términos del documento son: {integer_encoded}')
    
    letter = [0 for _ in range(len(words_index))]
    
    for value in integer_encoded:
        letter[value] = 1
        
    return letter

ohe(docu[0], vocab)

El index para cada término del vocabulario es: {'que': 0, 'de': 1, 'es': 2, 'gracias': 3, 'hoy': 4, 'el': 5, 'dia': 6, 'muchas': 7, 'martes': 8}
los términos del documento son: ['que', 'dia', 'es', 'hoy']
Los indexes de los términos del documento son: [0, 6, 2, 4]


[1, 0, 1, 0, 1, 0, 1, 0, 0]

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [196]:
from collections import Counter

def frequency(doc, vocab):
    
    words_index={}
    
    for word in vocab:
        word_no_punctuation = word.strip(string.punctuation)
        words_index[word_no_punctuation] = len(words_index)
    print(f'El index para cada término del vocabulario es: {words_index}')
    
    integer_encoded =[words_index[words] for words in doc]
    print(f'los términos del documento son: {doc}')
    print(f'Los indexes de los términos del documento son: {integer_encoded}')
    
    letter = [0 for _ in range(len(words_index))]
    
    for value in integer_encoded:
        letter[value] = Counter(integer_encoded)[value]
        
    return letter

frequency(docu[1], vocab)

El index para cada término del vocabulario es: {'que': 0, 'de': 1, 'es': 2, 'gracias': 3, 'hoy': 4, 'el': 5, 'dia': 6, 'muchas': 7, 'martes': 8}
los términos del documento son: ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes']
Los indexes de los términos del documento son: [8, 5, 6, 1, 4, 2, 8]


[0, 1, 1, 0, 1, 1, 1, 0, 2]

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [195]:
def TF_IDF(docus, vocab):
    
    ohe_docs = np.zeros([docus.size,len(vocab)])
    freq_docs = np.zeros([docus.size,len(vocab)])
    
    for i in range(len(docus)):
        ohe_docs[i,:] = ohe(docu[i], vocab)
        freq_docs[i,:] = frequency(docu[i], vocab)
    
    appear = np.sum(ohe_docs, axis= 0)
    
    idf = np.log10(docus.size/appear)
    tf = freq_docs
    print(f'tf: {tf}')
    print(f'idf: {idf}')
    
    tf_idf = tf*idf
    return tf_idf

TF_IDF(docu, vocab)        

tf: [[1. 0. 1. 0. 1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 0. 2.]
 [0. 0. 0. 1. 0. 0. 0. 1. 1.]]
idf: [0.47712125 0.47712125 0.17609126 0.47712125 0.17609126 0.47712125
 0.17609126 0.47712125 0.17609126]


array([[0.47712125, 0.        , 0.17609126, 0.        , 0.17609126,
        0.        , 0.17609126, 0.        , 0.        ],
       [0.        , 0.47712125, 0.17609126, 0.        , 0.17609126,
        0.47712125, 0.17609126, 0.        , 0.35218252],
       [0.        , 0.        , 0.        , 0.47712125, 0.        ,
        0.        , 0.        , 0.47712125, 0.17609126]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [194]:
import operator

def similitud(docus, index, vocab):
    
    tf_idf = TF_IDF(docus, vocab)
    
    similitudes = {i:cosine_similarity(tf_idf[i], tf_idf[index]) for i in range(len(tf_idf))}
    print(similitudes)
    sort = dict(sorted(similitudes.items(), key=operator.itemgetter(1), reverse=True))
    sim_sort = list(sort.keys())
    
    return sim_sort

similitud(docu, 1, vocab)

{0: 0.20034190268098703, 1: 1.0, 2: 0.10845711727883083}


[1, 0, 2]